In [37]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, PointDrawTool, CustomJS

output_notebook()

wh = 600

def rotate_point(p, center, angle_rad):
    R = np.array([[np.cos(angle_rad), -np.sin(angle_rad)],
                  [np.sin(angle_rad),  np.cos(angle_rad)]])
    return center + R @ (p - center)

def compute_all_rotations(x, y):
    p = np.array([x, y])
    angles = [np.pi/2, np.pi, 3*np.pi/2]
    rotated_points = []
    # Rotations around (1,1)
    for angle in angles:
        rotated_points.append(rotate_point(p, np.array([1,1]), angle))
    # Rotations around (0,0)
    for angle in angles:
        rotated_points.append(rotate_point(p, np.array([0,0]), angle))
    return np.array(rotated_points)

# Data sources
base_source = ColumnDataSource(data=dict(base_x=[0.5], base_y=[0.5]))
rot_source = ColumnDataSource(data=dict(rot_x=[], rot_y=[]))
line1_source = ColumnDataSource(data=dict(x=[], y=[]))
line2_source = ColumnDataSource(data=dict(x=[], y=[]))
line3_source = ColumnDataSource(data=dict(x=[], y=[]))

p = figure(x_range=(-1.1, 2.1), y_range=(-1.1, 2.1), match_aspect=True,
           width=wh, height=wh, title="Rotations & Perpendicular Bisectors")

# Plot base point
base_glyph = p.scatter('base_x', 'base_y', source=base_source, size=10, color='red', marker='circle')

# Plot rotated points (always show all 6)
rot_glyph = p.scatter('rot_x', 'rot_y', source=rot_source, size=8, color='blue', marker='circle')

# Plot bisector lines
p.line('x', 'y', source=line1_source, line_color='green', line_width=2)
p.line('x', 'y', source=line2_source, line_color='green', line_width=2)
p.line('x', 'y', source=line3_source, line_color='green', line_width=2)

p.line([1, 0], [0, 1], color='gray', line_width=2, alpha=0.8)
p.line([0, 1], [0, 1], color='gray', line_width=2, alpha=0.8)

# Unit square boundary
unit_square = [([0,1],[0,0]), ([1,1],[0,1]), ([1,0],[1,1]), ([0,0],[1,0])]
for xs, ys in unit_square:
    p.line(xs, ys, color='gray', line_width=2)

# p.legend.location = "top_left"

# Tool for dragging base point
draw_tool = PointDrawTool(renderers=[base_glyph], num_objects=1)
p.add_tools(draw_tool)
p.toolbar.active_drag = draw_tool

callback = CustomJS(args=dict(base_source=base_source, rot_source=rot_source,
                              line1_source=line1_source, line2_source=line2_source, line3_source=line3_source), code="""
    function clamp(v, min, max) {
        return Math.min(Math.max(v, min), max);
    }

    function rotate_point(px, py, cx, cy, angle) {
        let dx = px - cx;
        let dy = py - cy;
        let cosA = Math.cos(angle);
        let sinA = Math.sin(angle);
        return [cx + dx*cosA - dy*sinA, cy + dx*sinA + dy*cosA];
    }

    function midpoint(p1, p2) {
        return [(p1[0]+p2[0])/2, (p1[1]+p2[1])/2];
    }

    function perp_direction(p1, p2) {
        let dx = p2[0] - p1[0];
        let dy = p2[1] - p1[1];
        let len = Math.sqrt(dx*dx + dy*dy);
        return len === 0 ? [0,0] : [-dy/len, dx/len];
    }

    function intersect(p0, v0, p1, v1) {
        let A = [[v0[0], -v1[0]], [v0[1], -v1[1]]];
        let b = [p1[0]-p0[0], p1[1]-p0[1]];
        let det = A[0][0]*A[1][1] - A[0][1]*A[1][0];
        if (Math.abs(det) < 1e-8) return null;
        let t = (b[0]*A[1][1] - b[1]*A[0][1]) / det;
        return [p0[0] + t*v0[0], p0[1] + t*v0[1]];
    }

    // Clamp base point inside unit square
    let x = clamp(base_source.data.base_x[0], 0, 1);
    let y = clamp(base_source.data.base_y[0], 0, 1);
    base_source.data.base_x[0] = x;
    base_source.data.base_y[0] = y;

    // Compute all 6 rotated points (3 around (1,1), 3 around (0,0))
    let angles = [Math.PI/2, Math.PI, 3*Math.PI/2];
    let rotated = [];
    for (let angle of angles) {
        rotated.push(rotate_point(x, y, 1, 1, angle));
    }
    for (let angle of angles) {
        rotated.push(rotate_point(x, y, 0, 0, angle));
    }

    rot_source.data = {
        rot_x: rotated.map(p => p[0]),
        rot_y: rotated.map(p => p[1])
    };

    // Conditional bisector logic
    let line1_data = {x: [], y: []};
    let line2_data = {x: [], y: []};
    let line3_data = {x: [], y: []};

    if (x <= y) {
        // Use CCW around (1,1), CW around (0,0)
        let p0 = [x, y];
        let p1 = rotate_point(x, y, 1, 1, Math.PI/2);
        let p2 = rotate_point(x, y, 0, 0, -Math.PI/2);

        let origin1 = [1,1];
        let origin2 = [0,0];
        let origin3 = [1,0];

        function bisector_points(a, b, origin) {
            let mid = midpoint(a, b);
            let dir = perp_direction(a, b);
            // line param: origin + t * dir
            return {origin: origin, dir: dir};
        }

        let b1 = bisector_points(p0, p1, origin1);
        let b2 = bisector_points(p0, p2, origin2);
        let b3 = bisector_points(p1, p2, origin3);

        let inter = intersect(b1.origin, b1.dir, b2.origin, b2.dir);
        if (inter !== null) {
            inter = intersect(inter, [0,0], b3.origin, b3.dir) || inter;
            if (inter !== null) {
                line1_data = {x: [b1.origin[0], inter[0]], y: [b1.origin[1], inter[1]]};
                line2_data = {x: [b2.origin[0], inter[0]], y: [b2.origin[1], inter[1]]};
                line3_data = {x: [b3.origin[0], inter[0]], y: [b3.origin[1], inter[1]]};
            }
        }
    } else {
        // Use CW around (1,1), CCW around (0,0)
        let p0 = [x, y];
        let p1 = rotate_point(x, y, 1, 1, -Math.PI/2);
        let p2 = rotate_point(x, y, 0, 0, Math.PI/2);

        let origin1 = [1,1];
        let origin2 = [0,0];
        let origin3 = [0,1];

        function bisector_points(a, b, origin) {
            let mid = midpoint(a, b);
            let dir = perp_direction(a, b);
            return {origin: origin, dir: dir};
        }

        let b1 = bisector_points(p0, p1, origin1);
        let b2 = bisector_points(p0, p2, origin2);
        let b3 = bisector_points(p1, p2, origin3);

        let inter = intersect(b1.origin, b1.dir, b2.origin, b2.dir);
        if (inter !== null) {
            inter = intersect(inter, [0,0], b3.origin, b3.dir) || inter;
            if (inter !== null) {
                line1_data = {x: [b1.origin[0], inter[0]], y: [b1.origin[1], inter[1]]};
                line2_data = {x: [b2.origin[0], inter[0]], y: [b2.origin[1], inter[1]]};
                line3_data = {x: [b3.origin[0], inter[0]], y: [b3.origin[1], inter[1]]};
            }
        }
    }

    line1_source.data = line1_data;
    line2_source.data = line2_data;
    line3_source.data = line3_data;

    base_source.change.emit();
    rot_source.change.emit();
    line1_source.change.emit();
    line2_source.change.emit();
    line3_source.change.emit();
""")

base_source.js_on_change('data', callback)

show(p)

Loading BokehJS ...

In [93]:
import numpy as np

def compute_D(X):
    """
    X: np.ndarray, shape (2, n)
    Returns: D: np.ndarray, shape (n, n)
    """
    n = X.shape[1]

    A_all = np.stack([
        np.eye(2),                          # identity
        np.array([[0, -1], [1, 0]]),        # g1
        np.array([[-1, 0], [0, -1]]),       # g1^2
        np.array([[0, 1], [-1, 0]]),        # g1^3
        np.array([[-1, 0], [0, -1]]),       # g2
        np.array([[-1, 0], [0, -1]]),       # g3
        np.array([[0, -1], [1, 0]]),        # g4
        np.array([[-1, 0], [0, -1]]),       # g4^2
        np.array([[0, 1], [-1, 0]])         # g4^3
    ])  # shape (9, 2, 2)

    b_all = np.array([
        [0, 0],    # identity
        [0, 0],    # g1
        [0, 0],    # g1^2
        [0, 0],    # g1^3
        [2, 0],    # g2
        [0, 2],    # g3
        [2, 0],    # g4
        [2, 2],    # g4^2
        [0, 2]     # g4^3
    ]).T  # shape (2, 9)

    D = np.zeros((n, n), dtype=X.dtype)

    for ii in range(n):
        x = X[:, ii].reshape(2, 1)  # shape (2, 1)
        for jj in range(n):
            y = X[:, jj].reshape(2, 1)  # shape (2, 1)

            # Compute all 9 transformations: A @ y + b
            y_repeated = np.repeat(y[np.newaxis, :, :], 9, axis=0)  # (9, 2, 1)
            y_transformed = A_all @ y_repeated  # (9, 2, 1)
            y_transformed = y_transformed.squeeze(2) + b_all.T  # (9, 2)

            # Compute distances squared: ||x - y_transformed||^2
            diff = y_transformed - x.T  # (9, 2)
            dist_squared = np.sum(diff**2, axis=1)  # (9,)
            D[ii, jj] = np.min(dist_squared)

    return D

def create_X(n):
    """
    Creates a grid of points in the unit square, similar to the MATLAB version.
    
    Returns:
        X: np.ndarray of shape (2, n)
    """
    k = int(round(n ** 0.5))
    points = []

    for ii in range(1, k + 1):
        for jj in range(1, k + 1):
            x1 = (ii - 1) / k
            x2 = (jj - 1) / (k - 1) if k > 1 else 0
            points.append([x1, x2])

    X = np.array(points).T  # shape (2, n)
    return X

def compute_Xc(X):
    """
    Given X of shape (2, n), return X_c of shape (n,), where each complex number is x + i y.
    """
    return X[0] + 1j * X[1]


def process_complex(z):
    """
    Apply piecewise function to complex number z:
    - If Im(z) <= 1 - Re(z): return [sin(pi/8) * z**4, cos(pi/8) * |z|]
    - Else: reflect z across the line y = 1 - x, then apply same rule.

    Returns:
        np.ndarray of shape (2,), real values
    """
    # Check condition
    if z.imag > 1 - z.real:
        reflect = -1
        z, dist = reflect_across_y_eq_1_minus_x(z)
    else:
        _, dist = reflect_across_y_eq_1_minus_x(z)
        reflect = 1
    val1 = np.sin(np.pi / 8) * (z**4)
    val2 = np.cos(np.pi / 8) * abs(z)

    return np.array([val1, val2.real, reflect*dist])

def reflect_across_y_eq_1_minus_x(z):
    """
    Reflect complex number z across the line y = 1 - x
    """
    # Shift so line passes through origin
    c = 0.5 + 0.5j  # midpoint of the line segment [0,1] and [1,0]
    z_shifted = z - c

    # Reflect across line through origin with direction u
    u = (1 + 1j) / np.sqrt(2)

    proj_length = (z_shifted * np.conj(u)).real  # scalar projection
    proj_point = proj_length * u                 # complex projection point on the line
    z_reflected = z_shifted - 2 * proj_point

    dist = np.abs(z_shifted - z_reflected)/2

    # Shift back
    return z_reflected + c, dist

def compute_complex_distance_matrix(FXc):
    """
    FXc: np.ndarray of shape (n, 2), complex dtype
    Returns:
        D: np.ndarray of shape (n, n), real dtype
    """
    n = FXc.shape[0]
    D = np.zeros((n, n), dtype=np.float64)

    for i in range(n):
        for j in range(n):
            diff = FXc[i] - FXc[j]  # shape (2,)
            D[i, j] = (np.sum(np.abs(diff)**2))  # Euclidean norm

    return D

def entrywise_ratio_bounds(FD, D, tol=1e-12):
    """
    FD, D: np.ndarray of shape (n, n)
    tol: float, ignore entries where D < tol
    Returns: (min_ratio, max_ratio) over entries where D >= tol
    """
    mask = D >= tol
    ratio = np.zeros_like(FD, dtype=np.float64)
    ratio[mask] = FD[mask] / D[mask]
    
    if not np.any(mask):
        raise ValueError("All entries in D are below the tolerance threshold.")

    min_ratio = np.min(ratio[mask])
    max_ratio = np.max(ratio[mask])
    
    return min_ratio, max_ratio

In [104]:
X = create_X(225)  # Example usage to create a grid of points
D = compute_D(X)
Xc = compute_Xc(X)
FXc = np.array([process_complex(z) for z in Xc])  # shape (n, 2)
FD = compute_complex_distance_matrix(FXc)
min_ratio, max_ratio = entrywise_ratio_bounds(FD, D)
max_ratio/min_ratio

np.float64(673.8521381885109)

In [105]:
max_ratio

np.float64(3.269978949480359)

In [106]:
min_ratio

np.float64(0.004852665390764967)

In [95]:
Xc

array([0.        +0.j , 0.        +0.5j, 0.        +1.j , 0.33333333+0.j ,
       0.33333333+0.5j, 0.33333333+1.j , 0.66666667+0.j , 0.66666667+0.5j,
       0.66666667+1.j ])

In [96]:
D[3,:]

array([0.11111111, 0.02777778, 0.44444444, 0.        , 0.13888889,
       0.55555556, 0.11111111, 0.36111111, 0.88888889])

In [97]:
FD[3,:]

array([0.15041714, 0.03796709, 0.74443225, 0.        , 0.18896363,
       0.59986144, 0.155417  , 0.41118586, 0.88888889])

In [101]:
max_ratio/min_ratio

np.float64(9.83691898192851)

In [68]:
z = 0.5+0.5j
process_complex(z)  # Example usage of process_complex function

array([-0.09567086+0.j,  0.65328148+0.j])